In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Luminesce Syntax

Attributes
----------

"""

toggle_code("Toggle Docstring")


*Setting up the environment for the tutorial*

This section of the notebook creates a portfolio, some equities, and a file in drive to run luminesce queries against.

In [3]:
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
import lusid as lu
import pandas as pd
import lusid_drive
import lusid_drive.rest
import lusid.rest
from lusid_drive.rest import ApiException
from pprint import pprint
import json


secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.10274.0,0.5.3081,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [4]:
# here's some sample equities we've prepared earlier
ftse_instruments = pd.read_csv('_data/ftse100.csv')
nasdaq_instruments = pd.read_csv('_data/nasdaq100.csv')

In [5]:
# don't show exception if error is due to upsert
def exception_guard(e, code):
    return e.status and e.status != '400 Bad Request' and e.body and json.loads(e.body)["code"] == code

# create portfolio for this module
def upsert_portfolio(api_factory, scope, code, portfolio_name, portfolio_description):
    api_instance = api_factory.build(lu.TransactionPortfoliosApi)
        # Create an instance of the API class
    create_transaction_portfolio_request = {"displayName":portfolio_name,"description":portfolio_description,"code":code,"baseCurrency":"GBP", "instrumentScopes":["FBNUniversityModule-T2-2"]} # CreateTransactionPortfolioRequest | The definition of the transaction portfolio.
    try:
        # UpdatePortfolio: Update portfolio
        api_response = api_instance.create_portfolio(scope, create_transaction_portfolio_request)
        pprint(api_response)
    except lusid.rest.ApiException as e:
        if not exception_guard(e, 112):
            print("Exception when calling PortfoliosApi->update_portfolio: %s\n" % e)

scope = 'FBNUniversity' # str | The scope of the portfolio.
code = 'Module-T2-2' # str | The code of the portfolio. Together with the scope this uniquely identifies the portfolio.
portfolio_name = "Training module T2.2"
portfolio_description = "A portfolio to hold transactions"
upsert_portfolio(api_factory, scope, code, portfolio_name, portfolio_description)

In [6]:
# create a request to upsert a bunch of instruments.
def generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier):
    request_id = name + isin
    request ={"name":name,
                     "identifiers":{ "Figi":{"value":figi,"effectiveAt":"0001-01-01T00:00:00.0000000+00:00"},
                                     "Isin":{"value":isin,"effectiveAt":"0001-01-01T00:00:00.0000000+00:00"},
                                     other_identifier_name:{"value":other_identifier,"effectiveAt":"0001-01-01T00:00:00.0000000+00:00"}
                                   },

                     "definition":{"instrumentType":"Equity", "domCcy":ccy}}
    return request_id, request

In [7]:
# Create an instance of the API class
api_instance = api_factory.build(lu.InstrumentsApi)

# We're gonna load some american and UK equities into LUSID

other_identifier_name = 'Sedol'
ccy = 'GBP'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in ftse_instruments.itertuples())
ftse_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}
other_identifier_name = 'Cusip'
ccy = 'USD'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in nasdaq_instruments.itertuples())
nasdaq_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}

request_body_dict = nasdaq_body_dict | ftse_body_dict

try:
    # UpsertInstruments: Upsert instruments
    api_response = api_instance.upsert_instruments(request_body_dict, scope=scope)
#     pprint(api_response)
except lusid.rest.ApiException as e:
    print("Exception when calling InstrumentsApi->upsert_instruments: %s\n" % e)


In [8]:

# let's chuck a file containing a randomly selected subset of 25 instruments with their sectors into drive for some examples:

subset_file = open('_data/ftse_subset.csv').read()

drive_config = lusid_drive.Configuration(
    host = "https://fbn-uni.lusid.com/drive"
)
drive_config.access_token = RefreshingToken()

# Enter a context with an instance of the API client
with lusid_drive.ApiClient(drive_config) as api_client:
    # Create an instance of the API class
    api_instance = lusid_drive.FoldersApi(api_client)
    create_folder = {"path":"/","name":"finbourne university"} # CreateFolder | A CreateFolder object that defines the name and path of the new folder

    try:
        # [EARLY ACCESS] CreateFolder: Create a new folder in LUSID Drive
        api_response = api_instance.create_folder(create_folder)
        pprint(api_response)
    except ApiException as e:
        if not exception_guard(e, 664):
            print("Exception when calling FilesApi->create_file: %s\n" % e)
        
        
    api_instance = lusid_drive.FilesApi(api_client)
    x_lusid_drive_filename = 'ftse_subset.csv' # str | File name.
    x_lusid_drive_path = '/finbourne university/' # str | File path.
    content_length = len(subset_file.encode('UTF-8'))
    body = subset_file # str | 

    try:
        # [EARLY ACCESS] CreateFile: Uploads a file to Lusid Drive. If using an SDK, consider using the UploadAsStreamAsync function for larger files instead.
        api_response = api_instance.create_file(x_lusid_drive_filename, x_lusid_drive_path, content_length, body)
    except lusid_drive.rest.ApiException as e:
        if not exception_guard(e, 671):
            print("Exception when calling FilesApi->create_file: %s\n" % e)
        

# Luminesce Syntax

In this part of the course we will provide some additional technical information about luminesce, and demonstrate how luminesce queries can be used to extract information from your data.

## What is Structured Query Language (SQL)


SQL - pronounced "sequel" was first developed at IBM in the early 1970s, as a way to query and manipulate structured relational databases. SQL is a set-based, declarative programming language. Many implementations of SQL add extensions with procedural programming functionality, such as control-of-flow contructs.

## Luminesce Queries - SQL with extensions


SQLite is the most widely deployed database engine, used by several of the top web browsers, operating systems and application software.

Luminesce supports most of the query syntax for the SQLite dialect of SQL, but not the data manipulation or data definition syntax. Data can still be manipulated using some extensions to the language that Luminesce adds. 

Luminesce adds extensions to the SQLite implementation which can:
- exclude columns from queries
- automatically select important columns in queries
- parameterise queries
- write data to some Luminesce providers
- wait on results of some dependent operation, or for a time period
- run queries in parallel on an iterated range of parameters.



## Luminesce providers


A provider is a component that enables you to write a Luminesce SQL query for a data source in situ, without first having to extract, transform or load data from that source.

We supply providers for numerous data sources, including for the investment data stored in LUSID itself, for files stored in Drive and for some external data providers. You can also create your own provider for a proprietary data source.

There are two types of provider: a data provider and a direct provider.

A data provider is designed to query a data source whose shape is known. It therefore returns a table of results with a fixed number of fields (columns).

A direct provider is designed to query a data source whose shape is not known, and thus cannot return a table of results with a fixed number of fields (columns).

We'll use both data and direct providers in this tutorial.

## Use case: Using Luminesce to query and join data from disparate sources, and to distribute the resulting information.


I'd like to read a file from Drive and use the data from this file to filter transaction results. I'd like to join these filtered results with another provider and distribute the results. This will be packaged up into a simple to use View which runs on a schedule.

## An intro to SQLite queries

### A query using a simple SELECT statement


Its simple to get data from a provider using a select statement.

Use the following statement to get all data from a provider:

`SELECT * FROM SOME.PROVIDER`

Or retrieve data from a subset of columns by specifying them:

`SELECT A_COLUMN, ANOTHER_COLUMN FROM SOME.PROVIDER`

Here we'll use a SELECT statement to query for all of our porfolios.

In [9]:
%%luminesce
SELECT * 
FROM Lusid.Portfolio

,PortfolioScope,PortfolioCode,PortfolioType,BaseCurrency,DisplayName,Description,OriginPortfolioScope,OriginPortfolioCode,ParentPortfolioScope,ParentPortfolioCode,ChildPortfolioIds,CorporateActionSourceScope,CorporateActionSourceId,AccountingMethod,SubHoldingKeys,Created,InstrumentScopes
0,FBNUniversity,Module-1,Transaction,GBP,Module 1,NaN,FBNUniversity,Module-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2021-12-01,FBNUniversityModule-1
1,FBNUniversity,Module-T2-2,Transaction,GBP,Training module T2.2,A portfolio to hold transactions,FBNUniversity,Module-T2-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-11-03 20:14:15.334,FBNUniversityModule-T2-2
2,FBNUniversity,Module-3-2,Transaction,GBP,Training module 3.2,NaN,FBNUniversity,Module-3-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-3-2
3,FBNUniversity,Module-4-1,Transaction,GBP,Training module 4.1,NaN,FBNUniversity,Module-4-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-4-1
4,FBNUniversity,Module-6-2,Transaction,GBP,"Training module 6.2 - Same transaction code, d...",NaN,FBNUniversity,Module-6-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-2
5,FBNUniversity,Module-6-1,Transaction,GBP,Training module 6.1 - Different transaction co...,NaN,FBNUniversity,Module-6-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-1
6,FBNUniversity,Module-6-3,Transaction,GBP,Training module 6.3 - Reduce cash balance by c...,NaN,FBNUniversity,Module-6-3,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-3
7,FBNUniversity,Module-3-1,Transaction,GBP,Training module 3.1,NaN,FBNUniversity,Module-3-1,NaN,NaN,NaN,NaN,NaN,Default,Transaction/FBNUniversity/Module-3-1,2022-01-01,FBNUniversityModule-3-1
8,FBNUniversity,Module-6-4,Transaction,GBP,Training module 6.4 - Track commission as sepa...,NaN,FBNUniversity,Module-6-4,NaN,NaN,NaN,NaN,NaN,Default,Transaction/FBNUniversity/TrackCommission,2022-01-01,FBNUniversityModule-6-4


In this example, we've selected all columns and rows from the Lusid.Portfolio provider. We can specify column names if we are only interested in some parts of the data.

In [10]:
%%luminesce
SELECT DisplayName, PortfolioCode 
FROM Lusid.Portfolio

,DisplayName,PortfolioCode
0,Training module 3.1,Module-3-1
1,Training module 4.1,Module-4-1
2,Module 1,Module-1
3,Training module T2.2,Module-T2-2
4,"Training module 6.2 - Same transaction code, d...",Module-6-2
5,Training module 6.1 - Different transaction co...,Module-6-1
6,Training module 6.4 - Track commission as sepa...,Module-6-4
7,Training module 3.2,Module-3-2
8,Training module 6.3 - Reduce cash balance by c...,Module-6-3


Here I really just want to see the name of the portfolio and the portfolio code, I've selected only those columns from the provider.

### Limiting the number of return results using LIMIT


These simple SELECT statements are useful for getting an idea of the shape and quality of the data returned by a provider, but they tend to return a lot of data. Lets use LIMIT to reduce the amount of data requested:

In [11]:
%%luminesce
SELECT DisplayName, PortfolioCode 
FROM Lusid.Portfolio 
LIMIT 5

,DisplayName,PortfolioCode
0,Module 1,Module-1
1,Training module T2.2,Module-T2-2
2,Training module 3.2,Module-3-2
3,Training module 4.1,Module-4-1
4,"Training module 6.2 - Same transaction code, d...",Module-6-2


This example returns the first 5 rows of data from the Lusid.Portfolio provider.

### Filter queries WHERE ...


We can now filter these results down - lets get all of the Equities that are listed in GBP


In [12]:
%%luminesce
SELECT * 
FROM Lusid.Instrument.Equity 
WHERE DomCcy = 'GBP'

,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,CompositeFigi,ShareClassFigi,Wertpapier,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003D5N,NaN,NaN,NaN,NaN,NaN,BBG000NSXQ99,NaN,NaN,NaN,...,FBNUniversityModule-3-1,Burberry,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
1,LUID_00003DFI,GB00BKFB1C65,BKFB1C6,NaN,NaN,NaN,BBG00K9ZPS45,NaN,NaN,NaN,...,FBNUniversity,M&G,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
2,LUID_00003D96,GB0002405495,240549,NaN,NaN,NaN,BBG000BF0TF3,NaN,NaN,NaN,...,fbn-uni-T02002,Schroders,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
3,LUID_00003D4V,GB00B2B0DG97,NaN,NaN,REL LN,imd_43532542,BBG000D03XD4,NaN,NaN,NaN,...,FBNUniversityModule-1,RELX,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003D8F,JE00B6T5S470,B6T5S47,NaN,NaN,NaN,BBG0025RP8F9,NaN,NaN,NaN,...,fbn-uni-T02002,Polymetal International,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,LUID_00003DFW,GB0007669376,766937,NaN,NaN,NaN,BBG000BRMPC4,NaN,NaN,NaN,...,FBNUniversity,St. James's Place plc,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
262,LUID_00003DHX,GB0006731235,673123,NaN,NaN,NaN,BBG000BD3SC0,NaN,NaN,NaN,...,FBNUniversity,Associated British Foods,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
263,LUID_00003D9I,IE00BWT6H894,BWXC0Z1,NaN,NaN,NaN,BBG000DWL6M3,NaN,NaN,NaN,...,fbn-uni-T02002,Flutter Entertainment,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
264,LUID_00003DBN,GB00BLDYK618,BLDYK61,NaN,NaN,NaN,BBG000BFZMY9,NaN,NaN,NaN,...,fbn-uni-T02002,Scottish Mortgage Investment Trust,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP


We've filtered our query for equities who have a domestic currency of Pound Sterling.

### JOINing disparate datasources with join


Now, I've got a file in drive with sector information for some of these instruments. I'm going to join data from the file using our Drive provider with the saved instrument data so I can see the sectors for instruments where we know this information.

First lets get our instrument data.

Then we'll grab the file contents from drive.

We'll take a look at cross-joins, which SQLite uses to build left joins and inner joins. Cross-joins return the cartesian product of both tables - each row on the left table is concatenated with every row on the right table, to form a massive table with every combination of both tables' rows:


In [13]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector 
FROM Lusid.Instrument.Equity as Equities 
CROSS JOIN @x as File

,DisplayName,DomCcy,Sector
0,Maxim Integrated Products,USD,Financial Services
1,Maxim Integrated Products,USD,General Industrials
2,Maxim Integrated Products,USD,Travel & Leisure
3,Maxim Integrated Products,USD,Equity Investment Instruments
4,Maxim Integrated Products,USD,Household Goods & Home Construction
...,...,...,...
11795,Peloton Interactive,USD,Mining
11796,Peloton Interactive,USD,Media
11797,Peloton Interactive,USD,Mobile Telecommunications
11798,Peloton Interactive,USD,Food & Drug Retailers


We've returned a table with every combination of Lusid Equities with every row in our file.

In a left join, all rows from the left table are returned, along with rows in the right table that match the supplied key constraint. If there is a row in the left table with no matching row on the right, the columns that come from the right table in the resulting row are set to null.

In [14]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM Lusid.Instrument.Equity as Equities 
LEFT JOIN @x as File on Equities.Figi = File.Figi

,DisplayName,DomCcy,Sector,Figi
0,Pinduoduo Inc.,USD,NaN,BBG00LBLDDR2
1,"PepsiCo, Inc.",USD,NaN,BBG000C1X2B0
2,Burberry,GBP,NaN,BBG000NSXQ99
3,M&G,GBP,NaN,BBG00K9ZPS45
4,"Skyworks Solutions, Inc.",USD,NaN,BBG000GB4HW0
...,...,...,...,...
467,B&M,GBP,NaN,BBG006K4W740
468,Costco Wholesale Corporation,USD,NaN,BBG000F6H8W8
469,"Ross Stores, Inc.",USD,NaN,BBG000BSBZH7
470,"Synopsys, Inc.",USD,NaN,BBG000BSFRF3


We see this here as there are some companies with no matching sector in our file - these are returned as null.

We are going to ignore instruments without a matching sector for now, so we'll use an inner join.
Inner joins only return rows where the joining key has a match on both sides of the table:

In [15]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM Lusid.Instrument.Equity as Equities 
INNER JOIN @x as File on Equities.Figi = File.Figi

,DisplayName,DomCcy,Sector,Figi
0,Segro,GBP,Real Estate Investment Trusts,BBG000BF1HY7
1,Glencore,GBP,Mining,BBG001MM1KV4
2,BP,GBP,Oil & Gas Producers,BBG000C05BD1
3,Scottish Mortgage Investment Trust,GBP,Equity Investment Instruments,BBG000BFZMY9
4,DS Smith,GBP,General Industrials,BBG000BF1LF9
...,...,...,...,...
65,Intermediate Capital Group,GBP,Investment Services,BBG000D8KQJ7
66,Anglo American plc,GBP,Mining,BBG000BBLDF4
67,Schroders,GBP,Financial Services,BBG000BF0TF3
68,Melrose Industries,GBP,Automobiles & Parts,BBG00B5YJC26


Now we have a joined table where only Equities with a sector on file are returned, along with their sector.

### GROUPing results BY some condition in query


We can also group our results by a column - we'll perform an aggregation and group-by to count the number of instrument in each sector.

There are many different aggregation functions in SQLite, such as sum, avg and count.

Sum will return the sum of the specified field in each group.
Average returns the mean of the specified field in each group.
Count returns the number of results that are non-null for the field in each group. We can also specify that count returns the count of distinct values [Count(Distinct <field>)], or all rows [count(*)], in each group.

In [16]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT count(File.Sector), File.Sector
FROM Lusid.Instrument.Equity as Equities 
INNER JOIN @x as File on Equities.Figi = File.Figi
GROUP BY File.Sector

,count(File_Sector COLLATE BINARY),Sector
0,2,Automobiles & Parts
1,2,Equity Investment Instruments
2,4,Financial Services
3,2,Food & Drug Retailers
4,2,General Industrials
5,3,Household Goods & Home Construction
6,2,Investment Services
7,9,Life Insurance
8,2,Media
9,7,Mining


We've now grouped our results by sector, and for each sector, we''ve returned how many Equities belong to the sector.

## Introducing some Luminesce extensions


### Using ^ to SELECT the most important columns from a provider


Let's use the carat symbol to grab then more important columns in the Equity provider. The list of important columns is decided by the writer of the Provider.

In [17]:
%%luminesce
SELECT ^ 
FROM Lusid.Instrument.Equity

,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive
0,LUID_00003DH1,FBNUniversity,Maxim Integrated Products,Equity,Equities,USD,True
1,LUID_00003DD0,FBNUniversity,Monster Beverage Corporation,Equity,Equities,USD,True
2,LUID_00003DEJ,FBNUniversity,DCC plc,Equity,Equities,GBP,True
3,LUID_00003DBT,fbn-uni-T02002,Segro,Equity,Equities,GBP,True
4,LUID_00003D88,fbn-uni-T02002,HSBC,Equity,Equities,GBP,True
...,...,...,...,...,...,...,...
467,LUID_00003D5B,FBNUniversityModule-1,Burberry,Equity,Equities,GBP,True
468,LUID_00003D9Y,fbn-uni-T02002,Netflix,Equity,Equities,USD,True
469,LUID_00003DHN,FBNUniversity,Rio Tinto,Equity,Equities,GBP,True
470,LUID_00003DHQ,FBNUniversity,"Walgreen Boots Alliance, Inc.",Equity,Equities,USD,True


In the Lusid Equities provider: LusidInstrumentId, Scope, DisplayName, InferredAssetClass, InferredDomCcy and IsActive are the fields marked by the provider as being most important.

### Excluding columns from the results of a query using the except keyword.


Using some providers, we can also exclude columns from the result of a query by using the except keyword:

In [18]:
%%luminesce
SELECT ^ (except Type, IsActive) 
FROM Lusid.Instrument.Equity

,LusidInstrumentId,Scope,DisplayName,InferredAssetClass,InferredDomCcy
0,LUID_00003DA2,fbn-uni-T02002,Pinduoduo Inc.,Equities,USD
1,LUID_00003D7X,fbn-uni-T02002,"PepsiCo, Inc.",Equities,USD
2,LUID_00003D5N,FBNUniversityModule-3-1,Burberry,Equities,GBP
3,LUID_00003DFI,FBNUniversity,M&G,Equities,GBP
4,LUID_00003DBG,fbn-uni-T02002,"Skyworks Solutions, Inc.",Equities,USD
...,...,...,...,...,...
467,LUID_00003D6B,default,B&M,Equities,GBP
468,LUID_00003DHT,FBNUniversity,Costco Wholesale Corporation,Equities,USD
469,LUID_00003DFR,FBNUniversity,"Ross Stores, Inc.",Equities,USD
470,LUID_00003D9V,fbn-uni-T02002,"Synopsys, Inc.",Equities,USD


Here we select the most important columns again, but exclude the Type and IsActive columns

### Using parameters to filter queries


Many providers will have parameters that can also be used to filter a Luminesce query as part of a standard WHERE clause.

We can see these parameters by querying the Sys.Field Provider.

In [19]:
%%luminesce
SELECT FieldName, DataType, ParamDefaultValue, Description 
FROM Sys.Field 
WHERE TableName = 'Lusid.Instrument.Equity' 
AND FieldType = 'Parameter';

,FieldName,DataType,ParamDefaultValue,Description
0,AsAt,DateTime,0001-01-01T00:00:00Z,As at time to query at. Defaults to latest.
1,EffectiveAt,DateTime,0001-01-01T00:00:00Z,Effective time to query at. Defaults to latest.
2,UseLusidFilter,Boolean,True,Should the filter be translated to a Finbourne...


We can see there are 3 fields we can filter our query on: AsAt, EffectiveAt and UseLusidFilter

In [20]:
%%luminesce

SELECT * 
FROM Lusid.Instrument.Equity
WHERE UseLusidFilter = False

,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,CompositeFigi,ShareClassFigi,Wertpapier,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003DH1,US57772K1016,NaN,57772K101,NaN,NaN,BBG00JPW7422,NaN,NaN,NaN,...,FBNUniversity,Maxim Integrated Products,NaN,NaN,Equity,Equities,USD,True,NaN,USD
1,LUID_00003DD0,US61174X1090,NaN,61174X109,NaN,NaN,BBG008NVB1C0,NaN,NaN,NaN,...,FBNUniversity,Monster Beverage Corporation,NaN,NaN,Equity,Equities,USD,True,NaN,USD
2,LUID_00003DEJ,IE0002424939,242493,NaN,NaN,NaN,BBG000BFLBK3,NaN,NaN,NaN,...,FBNUniversity,DCC plc,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
3,LUID_00003DBT,GB00B5ZN1N88,B5ZN1N8,NaN,NaN,NaN,BBG000BF1HY7,NaN,NaN,NaN,...,fbn-uni-T02002,Segro,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003D88,GB0005405286,540528,NaN,NaN,NaN,BBG000BQ9HJ2,NaN,NaN,NaN,...,fbn-uni-T02002,HSBC,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,LUID_00003D5B,GB0031743007,NaN,NaN,BRBY LN,imd_43599356,BBG000NSXQ99,NaN,NaN,NaN,...,FBNUniversityModule-1,Burberry,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
468,LUID_00003D9Y,US64110L1061,NaN,64110L106,NaN,NaN,BBG000CL9VN6,NaN,NaN,NaN,...,fbn-uni-T02002,Netflix,NaN,NaN,Equity,Equities,USD,True,NaN,USD
469,LUID_00003DHN,GB0007188757,718875,NaN,NaN,NaN,BBG000D4Z9L5,NaN,NaN,NaN,...,FBNUniversity,Rio Tinto,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
470,LUID_00003DHQ,US9314271084,NaN,931427108,NaN,NaN,BBG000BWLMJ4,NaN,NaN,NaN,...,FBNUniversity,"Walgreen Boots Alliance, Inc.",NaN,NaN,Equity,Equities,USD,True,NaN,USD


Here we've filtered an Equity query on the UseLusidFilter. Notice there is no returned Field called UseLusidFilter - its not a column in our data.

## Using Luminesce variables


You can use variables to create and populate arbitrary tables of data that can then be used as part of a Luminesce query for either a data provider or a direct provider.

Declare a variable that represents a table of data using:
`@variable`

Or to represent a single value:
`@@variable`

Let's see variables in action. 

In [21]:
%%luminesce
@instruments_in_GBP = SELECT * 
                      FROM Lusid.Instrument.Equity 
                      WHERE DomCcy = 'GBP';
SELECT Isin 
FROM @instruments_in_GBP;


,Isin
0,GB00BKFB1C65
1,GB0002405495
2,GB00B2B0DG97
3,JE00B6T5S470
4,GB00BH4HKS39
...,...
249,GB0007669376
250,GB0006731235
251,IE00BWT6H894
252,GB00BLDYK618


Here_instruments_in GBP is a table variable, set from the results of the query on equities.

In [22]:
%%luminesce
@sectors = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

@@selected_sector = SELECT 'Life Insurance';
SELECT * 
FROM @sectors 
WHERE Sector = @@selected_sector

,Name,Sector,Figi
0,Prudential plc,Life Insurance,BBG000BDY322
1,St. James's Place plc,Life Insurance,BBG000BRMPC4
2,Legal & General,Life Insurance,BBG000BDQCG6
3,Aviva,Life Insurance,BBG000DDCSJ1


I've set selected_sector as a single-value variable "Lise Insurance", and filtered my file by sector.

## Introducing more advanced Luminesce extensions


You can use the CROSS APPLY or OUTER APPLY statement to iterate over a set of parameters. This is useful to execute a Luminesce query in parallel for each of the parameter inputs.

### Cross-apply

Cross-apply is similar to INNER JOIN, in that only records that have matching values on both sides are returned. 
We'll run some equity queries in parallel using Tools.split to create a set of currencies to filter by- the table will contain the currency joined to all of the returned equity tables:

In [23]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';

SELECT c.val as selectedCurrency, results.*
FROM @currencies c
CROSS APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity equity
       WHERE equity.DomCcy = c.val
       
) results;

,selectedCurrency,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003DEJ,FBNUniversity,DCC plc,Equity,Equities,GBP,True,1
1,GBP,LUID_00003DBT,fbn-uni-T02002,Segro,Equity,Equities,GBP,True,1
2,GBP,LUID_00003D88,fbn-uni-T02002,HSBC,Equity,Equities,GBP,True,1
3,GBP,LUID_00003D8L,fbn-uni-T02002,RELX,Equity,Equities,GBP,True,1
4,GBP,LUID_00003D4W,FBNUniversityModule-1,Halma,Equity,Equities,GBP,True,1
...,...,...,...,...,...,...,...,...,...
467,USD,LUID_00003D99,fbn-uni-T02002,Idexx Laboratories,Equity,Equities,USD,True,2
468,USD,LUID_00003DHT,FBNUniversity,Costco Wholesale Corporation,Equity,Equities,USD,True,2
469,USD,LUID_00003DFR,FBNUniversity,"Ross Stores, Inc.",Equity,Equities,USD,True,2
470,USD,LUID_00003D9V,fbn-uni-T02002,"Synopsys, Inc.",Equity,Equities,USD,True,2


This example works similar to an inner join, we've run a query in parallel for some currency values, returning a similar table to an inner join. Remember, inner join returns only rows which fulfil the join criteria.

### Outer-apply


Outer apply is similar to LEFT OUTER JOIN, in that all records on the left side are returned, even when there are no matching records on the right side.

In [24]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';
SELECT c.val, results.*
FROM @currencies c
OUTER APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity as equity
       WHERE equity.DomCcy = c.val
) results
WHERE val == CHF

,val,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003D4V,FBNUniversityModule-1,RELX,Equity,Equities,GBP,True,1.0
1,GBP,LUID_00003D4W,FBNUniversityModule-1,Halma,Equity,Equities,GBP,True,1.0
2,GBP,LUID_00003D4X,FBNUniversityModule-1,Unilever,Equity,Equities,GBP,True,1.0
3,GBP,LUID_00003D4Y,FBNUniversityModule-1,Just Eat Takeawy,Equity,Equities,GBP,True,1.0
4,GBP,LUID_00003D4Z,FBNUniversityModule-1,Mondi,Equity,Equities,GBP,True,1.0
...,...,...,...,...,...,...,...,...,...
468,USD,LUID_00003DHY,FBNUniversity,JD.com,Equity,Equities,USD,True,2.0
469,USD,LUID_00003DI0,FBNUniversity,Fox Corporation (Class A),Equity,Equities,USD,True,2.0
470,USD,LUID_00003DI1,FBNUniversity,Electronic Arts,Equity,Equities,USD,True,2.0
471,USD,LUID_00003DI5,FBNUniversity,Keurig Dr Pepper Inc.,Equity,Equities,USD,True,2.0


We've run a query in parallel for some currency values, returning a similar table to a left join with our selected currencies on the left side of the join. Here, the values from Lusid.Instrument.Equity on the CHF row are null.